# Gauss Code for "A Fixed-b Perspective on Inference for High Frequency Financial Data"

## Main Simulation

In [ ]:
new;
n_table = 5;
ku = seqa(0,1.5,n_table);
kx = seqa(0,1.5,n_table);
table_NW_mean = zeros(n_table,n_table);
table_AD_mean = zeros(n_table,n_table);
table_NWband_mean = zeros(n_table,n_table);
table_ADband_mean = zeros(n_table,n_table);
for w (1, rows(ku),1);
    for d (1, rows(kx),1);
        NW_set = 0;
        NW_band_set = 0;
        AD_set = 0;
        AD_band_set = 0;
        clear NW_set;
        clear NW_band_set;
        clear AD_set;
        clear AD_band_set;
        for i (1,100,1);
            rndseed i;
            u = OU(0,30,30*252,ku[w],0,0.0097);
            x = OU(0,30,30*252,kx[d],0,0.0998);
            b0 = 0;
            b1 = 1;
            y = b0 + b1*x + u;
            nu = rows(y);
            cons1 = ones(nu,1);
            X2 = cons1 ~ x;
            coeffs = invpd(X2'X2)*X2'y;
            X_hat = X2*coeffs;
            uhat = y - X_hat; @G-test@
            vhat = uhat;
            R = { 1 0, 0 1 };
            r2 = {0,1};
            NW1 = Gtest(R,r2,coeffs,X2,vhat,5.5*0.275,7/25,2,2); @NW is bandtype 2@
            NW2 = NW_band(vhat,5.5*0.275,7/25,2);
            NW_set = NW_set~NW1;
            NW_band_set = NW_band_set~NW2;
            AD1 = Gtest(R,r2,coeffs,X2,vhat,5.5*0.275,7/25,2,1); @AD is bandtype 1@
            AD2 = AD_band(vhat);
            AD_set = AD_set~AD1;
            AD_band_set = AD_band_set~AD2;
        endfor;
        table_NW_mean[w,d] = meanc(NW_set[2:cols(NW_set)]');
        table_AD_mean[w,d] = meanc(AD_set[2:cols(AD_set)]');
        table_NWband_mean[w,d] = meanc(NW_band_set[2:cols(NW_band_set)]');
        table_ADband_mean[w,d] = meanc(AD_band_set[2:cols(AD_band_set)]');
    endfor;
endfor;

#### Autocovariance estimator
$$\gamma_{T}(j) = T^{-1}\sum^{T}_{t=j+1}{\hat{v}_{i}\hat{v}_{i-j}} = \frac{1}{T}\hat{V}_{(i)}'\hat{V}_{(i-j)}$$
and $\hat{v}_{i} = \hat{u}_{i} $ or  $x_{i}\hat{u}_{i}$

In [2]:
proc(1) = autocov_est(vhat,j);
    local T,v_t,v_tm1,gamma_hat_j;
    if j >= 0;
        T = rows(vhat);
        v_t = vhat[j+1:T,.];
        v_tm1 = vhat[1:T-j,.];
        gamma_hat_j = v_t'v_tm1/T;
        retp(gamma_hat_j);
    else;
        @ if j < 0 @
        T = rows(vhat);
        v_t = vhat[-j+1:T,.];
        v_tm1 = vhat[1:T+j,.];
        gamma_hat_j = v_t'v_tm1/T;
        retp(gamma_hat_j);
    endif;
endp;

#### Newey-West Bandwidth
$$\hat{\sigma}_{i} = T^{-1}\sum_{t = i+1}^{T}{w'\hat{v}_{t}\hat{v}_{t-i}'w}$$
Newey-West compute nonparametric truncated kernel estimators as follows
$$\hat{f}^{(q)} = \frac{1}{2\pi}\sum_{j=-a_{n}}^{a_{n}}{|j|^{q}}\hat{\sigma}_{j}$$
where $a_{n} = cn^{p}$ is Newey-West lag selection parameter and $p = 4/25$ for Prazen kernel.

Then, we have
$$\hat{\alpha}(2) = (\hat{f}^{(q)}/\hat{f}^{(0)})^{2}$$

$$b_{T}^{*} = c_{k}[\hat{\alpha}(q)T]^{\frac{1}{2q+1}}$$

In [ ]:
proc(1) = NW_band(vhat,c,p,q);
    local n,T,j,nw_lag,f_q,f_0,a_n,a_hat_2,ST_NW,w,wp;
    n = rows(vhat);
    T = rows(vhat);
    if cols(vhat) == 1;
        nw_lag = c*(n^p);
        a_n = nw_lag;
        f_q = 0;
        f_0 = 0;
        for j (-int(a_n),int(a_n),1);
            f_q = f_q + (abs(j)^q)*autocov_est(vhat,j);
            f_0 = f_0 + autocov_est(vhat,j);
        endfor;
        a_hat_2 = (f_q/f_0)^2;
        ST_NW = 2.6614*(T*a_hat_2)^(0.2); @bandwidth@
        retp(ST_NW);
      
      elseif cols(vhat) == 2;  @ n by 2 matrix @
        nw_lag = c*(n^p);
        a_n = nw_lag;
        f_q = 0;
        f_0 = 0;
        wp = {0 1};
        w = wp';
        for j (-int(a_n),int(a_n),1);
            f_q = f_q + (abs(j)^q)*((wp*autocov_est(vhat,j))*w);
            f_0 = f_0 + ((wp*autocov_est(vhat,j))*w);
        endfor;
        a_hat_2 = (f_q/f_0)^2;
        ST_NW = 2.6614*(T*a_hat_2)^(0.2);
        
        retp(ST_NW);
    endif;
endp;

#### Parzen Kernel

In [ ]:
proc(1) = Parzen(x);
    local y;
    y=(x.>=0.0).*(x.<=0.5).*(1.0-6.0*x.*x.*(1.0-x));
    y=y+(x.>0.5).*(x.<=1.0)*2.0.*(1.0-x).^3;
    retp (y);
endp;

### Longrun variance estimator
$$\hat{\Omega}_{T}^{2} = \sum_{|j|\leq T-1}K(\frac{j}{b_{T}^{*}})\hat{\gamma}_{T}(j)$$

In [ ]:
proc(1) = LRV(vhat,c,p,q,bandtype);
    local T, LRV2, b_n, j;
    if bandtype == 1; @2 is NW, 1 is AD@
        T = rows(vhat);
        LRV2 = 0;
        b_n = AD_band(vhat);
        for j (-int(T)+1,int(T)-1,1);
            LRV2 = LRV2 + Parzen(j/b_n)*autocov_est(vhat,j);
        endfor;    
        
    elseif bandtype == 2; @2 is NW, 1 is AD@
        T = rows(vhat);
        LRV2 = 0;
        b_n = NW_band(vhat,c,p,q);
        for j (-int(T)+1,int(T)-1,1);
            LRV2 = LRV2 + Parzen(j/b_n)*autocov_est(vhat,j);
        endfor;
    endif;
    retp(LRV2);
endp;

## Andrews Bandwidth

In [ ]:
proc(1) = AD_band(vhat);
    local T,vhat_t,vhat_tm1,rho_hat,alpha_hat2,ST;
    if cols(vhat) == 2;
        vhat = vhat[.,2];
    endif;
    T = rows(vhat);
    vhat_t = vhat[2:T,.];
    vhat_tm1 = vhat[1:T-1,.];
    rho_hat = vhat_t/vhat_tm1;
    
    if abs(rho_hat - 1) < 0.001;
        rho_hat = 1 + 0.001*((rho_hat-1)/abs(rho_hat-1));
    endif;
    
    alpha_hat2 = (4*(rho_hat^2))/((1-rho_hat)^4);
    ST = 2.6614*(T*alpha_hat2)^0.2;
    retp(ST);
endp;

## OU Processes

In [ ]:
proc(1) = OU(t_0,t_end,length,theta,mu,sigma);
    local t,dt,y, noise,i;
    t = seqa(t_0,(t_end-t_0)/(length-1),length);
    dt = (t_end-t_0)/(length-1);
    y = zeros(length,1);
    noise = rndn(length,1)*sqrt(dt);
    for i (2, length,1);
        y[i] = y[i-1] + theta*(mu-y[i-1])*dt + sigma*noise[i];
    endfor;
    retp(y);
endp;

## G-test

In [ ]:
proc(1) = Gtest(R,r2,b,X2,vhat,c,p,q,bandtype);
    local j2,Xv,s,G_pre,G_stat;
    if bandtype == 1; @2 is NW, 1 is AD@
        j2 = R*b - r2;
        Xv = invpd(X2'X2);
        s = (R*Xv)*R';
        G_pre = (j2'*inv(s))*j2;
        
        G_stat = G_pre/LRV(vhat,c,p,q,bandtype); 
    
    elseif bandtype == 2; @2 is NW, 1 is AD@
        j2 = R*b - r2;
        Xv = invpd(X2'X2);
        s = (R*Xv)*R';
        
        G_pre = (j2'*inv(s))*j2;
        
        G_stat = G_pre/LRV(vhat,c,p,q,bandtype);
    endif;
    retp(G_stat);
endp;     